In [ ]:
import cv2
import csv
import sys
import requests
import skimage.io
import os
import os.path

import glob
import pickle
import time

from keras.applications import VGG16
import datetime

from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image as kimage
import numpy as np
import pandas as pd
import scipy.sparse as sp

import skimage.io
import hashlib


import matplotlib.pyplot as plt

from tensorflow.python.client import device_lib

import tensorflow as tf


import math
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from shutil import copyfile

from PIL import Image as pimage, ExifTags
from PIL.ExifTags import TAGS

from similaridade import *
%matplotlib inline

print("inicio")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

gera_arqs = True
gera_dados = True
sim_tensorflow = True

arqs = {}
arqs["busca"] = {}
arqs["material"] = {}
arqs["extensoes_img"] = ['.jpg', '.png', '.tif', '.jpeg', '.gif', '.bmp']

arqs["busca"]["raiz"] = "./img_busca/"
arqs["busca"]["nome_exibicao"] = "CD Anexo"
arqs["busca"]["nomes"] = []
arqs["busca"]["hashs"] = {}

arqs["material"]["raiz"] = "./img_quest/"
arqs["material"]["nome_exibicao"] = "HD Examinado"
arqs["material"]["nomes"] = []
arqs["material"]["hashs"] = {}

if gera_arqs:
    print("\ngera arqs")
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    get_arquivos_hashs("busca", arqs)
    get_arquivos_hashs("material", arqs)
    grava_arqs(arqs)
else:
    print("\nle arqs")
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    arqs = le_arqs()

print("busca keys = ", len(arqs["busca"]["hashs"].keys() ) )
print("busca nomes = ",len(arqs["busca"]["nomes"]))
print("material keys = ", len(arqs["material"]["hashs"].keys() ) )
print("material nomes = ",len(arqs["material"]["nomes"]))

#print("\nestrutura arqs")
#print_arqs(arqs)

print("\n\ntestes...")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

rand_img = os.path.join(arqs["busca"]["raiz"] ,np.random.choice(arqs["busca"]["nomes"]))
print(rand_img)
print(os.path.isfile(rand_img))

try:
    img = skimage.io.imread(rand_img)
    print(img.shape)
except:
    print("erro de leitura do arquivo", rand_img)  
    
img = kimage.load_img(rand_img, target_size=(224, 224))
x = kimage.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print(x.shape)

print("\ncarrega vgg16")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
# image_top=False removes final connected layers
model = VGG16(include_top=False, weights='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5')

pred = model.predict(x)
print(pred.shape)
print(pred.ravel().shape)

pred = model.predict(x)

print("\nmonta dados")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))

dados = {}
dados["busca"]    = {}
dados["material"] = {}
dados["total"]    = {}

dados["busca"]["itens"] = []
dados["busca"]["nomes"] = []

dados["material"]["itens"] = []
dados["material"]["nomes"] = []

dados["total"]["itens"] = []
dados["total"]["nomes"] = []

dados["predicoes"]    = None
dados["predicoescsr"] = None
dados["sim"]          = None

dados["busca"]["itens"]=list(arqs["busca"]["hashs"].items())
dados["busca"]["nomes"] = [os.path.join(arqs["busca"]["raiz"] ,f[1][0]) for f in dados["busca"]["itens"]]

dados["material"]["itens"]=list(arqs["material"]["hashs"].items())
dados["material"]["nomes"] = [os.path.join(arqs["material"]["raiz"] ,f[1][0]) for f in dados["material"]["itens"]]

dados["total"]["itens"] = dados["busca"]["itens"] + dados["material"]["itens"]
dados["total"]["nomes"] = dados["busca"]["nomes"] + dados["material"]["nomes"]

#print("\nestrutura dados")
# estrutura de dados
#print_dados(dados)

print("\ncheca dados")
    
#sanity check - se imprimir alguma coisa tem algo errado
for i in range(0, len(dados["busca"]["itens"])):
    x1 = dados["busca"]["itens"][i]
    x2 = dados["busca"]["nomes"][i]
    y1 = dados["total"]["itens"][i]
    y2 = dados["total"]["nomes"][i]
    if (x1[0]!=y1[0]): print(x1[0], y1[0])
    if (x1[1]!=y1[1]): print(x1[1], y1[1])
    if (x2!=y2): print(x1, y2)
#limites
print(dados["total"]["nomes"][i])
print(dados["total"]["nomes"][i+1])
print(dados["total"]["itens"][i])
print(dados["total"]["itens"][i+1])


batch_size = 500
min_idx = 0
total_max = len(dados["total"]["nomes"])
max_idx = min(min_idx + batch_size, total_max)
n_dims = pred.ravel().shape[0]
px = 224

# matriz de predições
dados["predicoes"] = sp.lil_matrix((len(dados["total"]["nomes"]), n_dims))

if gera_dados:
    print("\ngera predições")
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    while min_idx < total_max - 1:

        X = np.zeros(((max_idx - min_idx), px, px, 3))

        # carrega arquivos em X
        for i in range(min_idx, max_idx):
            fname = dados["total"]["nomes"][i]
            imgcv2 = cv2.imread(fname)
            imgcv2 = cv2.cvtColor(imgcv2, cv2.COLOR_BGR2RGB)
            imgcv2 = cv2.resize(imgcv2, (px, px))
            #fig=plt.figure()
            #plt.imshow(imgcv2)
            #img = pimage.fromarray(imgcv2)
            img = imgcv2
            #img = img.resize((px, px), pimage.ANTIALIAS)
            # transforma imagem num array e carrega em X
            img_array = kimage.img_to_array(img)
            X[i - min_idx, :, :, :] = img_array

        # pre processa X (usa função keras.applications.vgg16.preprocess_input)
        X = preprocess_input(X)
        these_preds = model.predict(X)
        shp = ((max_idx - min_idx), n_dims) #shp = ((max_idx - min_idx) + 1, n_dims)

        dados["predicoes"][min_idx:max_idx, :] = these_preds.reshape(shp) #dados["predicoes"][min_idx:max_idx + 1, :] = these_preds.reshape(shp)

        min_idx = max_idx
        max_idx = np.min((max_idx + batch_size, total_max))
    print("\ngrava dados predições")
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    grava_dados_pred(dados)
else:
    print("\nlê dados predições")
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    dados = le_dados_pred() 

print("\ndevices")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
print(get_available_devices())  

    
if sim_tensorflow:
    print("\ncalcula similaridades usando tensorflow")
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    dados["sim"] = comp_similarity_tf2(dados["predicoes"])
else:
    print("\ncalcula similaridades usando apenas numpy")
    print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    # transforma as predicoes em uma compressed sparse row (csr)
    dados["predicoescsr"] = dados["predicoes"].tocsr()
    dados["sim"] = comp_similarity(dados["predicoescsr"])

print("\nrelatorio busca")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
excl = [ ]
limiar = []
#nivel = 0.2
nro_arq_busca = 0
gera_html_busca(dados, arqs, excl, N=3, minimoinicial=0.20, minimofinal=0.2)
print(limiar)

print("\nrelatorio material")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
excl = [ ]
limiar = []
nro_arq_busca = 0
gera_html_material(dados, arqs, excl, N=3, minimoinicial=0.2, minimofinal=0.2)
print(limiar)
print("Fim")
print(datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
